In [40]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.batchrunner import BatchRunner
import matplotlib.pyplot as plt
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import itertools
from collections import deque
from itertools import starmap,product
import collections

class MemoryDeque(deque):
    def __init__(self,liste,maxlen):
        super().__init__(liste,maxlen)
    def insert_and_pop(self,index,what):
        try:
            self.insert(index,what)
        except IndexError:
            self.pop()
            self.insert(index,what)

class FoodAgent(Agent):
    def __init__(self,unique_id,model,boja,ukus,oblik):
        super().__init__(unique_id,model)
        self.boja = boja
        self.oblik = oblik
        self.ukus = ukus
        self.size = 0.5
    def step(self):
        pass
#duzina memorije, broj agenata, velicina table, gubitak energije, 
#ako imamo 10X10 tablu, imamo ukupno 64 kombinacije
#ustvari oni treba da se preslikavaju na broj tako da ce biti ok sve
#znaci struktura je [((plavo,zvezda,ljuto),-1),....))]
#search nece biti bas najbolji, ali sta cemo, da je dictionary, ne bismo mogli da
#---------------------------------------------------------
#|  -4      |      +4      |     -3       |      +3      |
#---------------------------------------------------------
#|    -2    |       +2     |      -1      |     +1       |
#---------------------------------------------------------
#pozicije -4 -> 0,+4->3,...
#TODO - napraviti sve robusnije, da mogu ovi svi parametri da menjaju. Znaci kompleksnost sveta, i broj hrana koji 
#cemo uzeti iz tog sveta. 
#kada zavrsim onda istestiramo ! 
#TODO prebaci u datatapyes.py
komb_tuple = collections.namedtuple("Kombinacija","boja ukus oblik")
memory_tuple = collections.namedtuple("Memorija","kombinacija hranljivost")
#ovde ce zavisiti od broja ovih stvari, znaci broj memorije mi ostaje isti, 32, ili ne, uglavnom ce se deliti na osam, tako da ali da bude memorija uvek x8, za sada
#32/8 = 4, pozicije su 0,4,8,12. a stepeni otrova ih je 8, ako bude 64/8 =
#velicina podeoka u memoriji=len(memorija)/br_stepena_otrovnosti
#pozicija = 0
# for i in range(len(memorija),step=4):
#    pozicije_u_proceduralnoj[otrovnost]=pozicija
# pozicija+=vel_podeoka

#pozicije_u_proceduralnoj = {-4:0,
#                      +4:4,
#                      -3:8,
#                      +3:12,
#                      -2:16,
#                      +2:20,
#                      -1:24,
#                      +1:28}
 #ovo menjamo, ovo dole funkcionise

#sve_kombinacije = list(itertools.product(boje,ukusi,oblici))







def izdvoji_agente(model):
    return [a for a in model.schedule.agents if isinstance(a,HungryAgent)]

def compute_knowledge(model):
    """Ukupno znanje svih HungryAgenata"""
    agents = izdvoji_agente(model)
    ukupno = 0
    for agent in  agents:
        for key,value in agent.svet.items():
            assert(len(value)<=4)
            ukupno+=len(value)
    return ukupno

def total_pojedeni_otrovi(model):
    """Vraca broj ukupnih pojedenih otrova"""
    agents = izdvoji_agente(model)
    ukupno = 0
    for agent in agents:
        ukupno+=agent.pojedeni_otrovi
    return ukupno

def total_pojedena_hrana(model):
    """Vraca broj ukupnih pojedenih otrova"""
    agents = izdvoji_agente(model)
    ukupno = 0
    for agent in agents:
        ukupno+=agent.pojedena_hrana
    return ukupno

def measure_experience(model):
    """Ukupno iskustvo"""
    agents = izdvoji_agente(model)
    ukupno = 0
    for agent in agents:
        ukupno+=len(agent.procedural)
    return ukupno

def total_energy(model):
    """Meri energiju svih agenata"""
    agents = izdvoji_agente(model)
    ukupno = 0
    for agent in agents:
        ukupno+=agent.energy
        
    return ukupno

class HungryAgent(Agent):
    def __init__(self,unique_id,model,memory_size,br_stepena_otrovnosti,walk_energy):
        super().__init__(unique_id,model)
        self.energy = 0
        self.walk_energy=0.2
        self.svet = {"boje":set(),"oblici":set(),"ukusi":set()}
        self.boja = "black"
        self.size = 0.2
        self.procedural =MemoryDeque([],maxlen=memory_size)
        self.pojedeni_otrovi=0
        self.pojedena_hrana=0
        self.insertion_dict = self.make_insertion_dict(br_stepena_otrovnosti= br_stepena_otrovnosti,memory_size=memory_size)
        
    def move(self):
        self.energy=-self.walk_energy
        possible_steps = self.model.grid.get_neighborhood(self.pos,moore=True,include_center = False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self,new_position)
        
    def store_info(self,food):
        self.svet["boje"].add(food.boja)
        self.svet["oblici"].add(food.oblik)
        self.svet["ukusi"].add(food.ukus)
        
    def make_insertion_dict(self,br_stepena_otrovnosti,memory_size):
        vel_podeoka = memory_size//br_stepena_otrovnosti
        poz = dict()
        pozicija = 0
        for i in range(-(br_stepena_otrovnosti//2),0):
            poz[i]=pozicija
            pozicija+=vel_podeoka
            poz[-i]=pozicija
            pozicija+=vel_podeoka
        return poz

        
    def eat_or_not(self,food):

        def search_memory_deque(mdeque,ktuple):
            for entry in mdeque:
                print (entry)
                if entry.kombinacija ==ktuple:
                    print ("nadjeno u memoriji!")
                    return entry
            return None
                
                
                
        eat = None
        property_tuple=komb_tuple(boja=food.boja,ukus=food.ukus,oblik=food.oblik)
        
        rezultat = search_memory_deque(self.procedural, property_tuple)
        
        if rezultat:
            eat = False if rezultat.hranljivost<0 else True
        else:
            #barf, scrape_memory
            assert(eat is None)
            hranljivost=0
            for atribut in property_tuple:
                # za svaki atribut cemo skrejpovati memoriju, u svim mogucim kombincijama(mozda da napravim poseban dict, koji broji koliko puta je koji false, koliko puta je koji True. svakako bi se brojevima svasta resilo,takodje i stepni otrova, direkno ih mogu sabirati umesto svih ifova.)
                for ktuple in self.procedural:
                    if atribut in ktuple.kombinacija:
                        hranljivost+=ktuple.hranljivost
            
            print ("HRANLJIVOST: %s" %hranljivost)
            if hranljivost>0:
                eat=True
            elif hranljivost<0:
                eat=False
            else:
                eat = True #self.random.choice([True,False])
        
            if eat:
                hranljivost=self.model.food_dict[property_tuple]
                print (hranljivost)
                poz = self.insertion_dict[hranljivost]
                self.procedural.insert_and_pop(poz,memory_tuple(kombinacija=property_tuple,hranljivost=hranljivost))
                #ovde smo jedino ako nismo nasli u dictu
                        
        if eat:     #na kraju gledamo jesmo li pojeli ili nismo i onda sabiramo (posto smo mozda vec na pocetku pojeli) mozda ovde u explore npr da stavimo
            self.energy+=self.model.food_dict[property_tuple] #dobicemo ili oduzecemo energiju
            if self.model.food_dict[property_tuple]<0:
                self.pojedeni_otrovi+=1
            elif self.model.food_dict[property_tuple]>0:
                self.pojedena_hrana+=1
               
        
        
    def explore(self):
        """Ide negde i gleda na sta je nagazio"""
        foods = [f for f in self.model.grid.get_cell_list_contents([self.pos]) if isinstance(f,FoodAgent)]
        for food in foods:
            self.store_info(food)
            self.eat_or_not(food)
            
        

    def step(self):
       # print (self.unique_id)
       # print("Imam:%s dinara" %self.wealth)
        self.move()
        self.explore()
        #moze da komunicira sa okruzenjem i da saznaje o drugim bicima
        
        
    
class HungerModel(Model):
    """A model with some number of agents."""
    boje = ["red","yellow","green","blue"]
    ukusi = ["kiselo","ljuto","gorko","slatko"]
    oblici = ["zvezda","kvadrat","krug","trougao"]
    sve_kombinacije= list(starmap(komb_tuple,product(boje,ukusi,oblici)))
    def __init__(self,N,width,height,num_of_food=64,br_stepena_otrovnosti=8,agent_memory_size=32,agent_walk_energy=0.2):
        #ovih num_of_food random bira TODO
        self.kombinacije = self.sve_kombinacije[:num_of_food]
        self.food_dict = self.raspodeli_hranu(self.kombinacije,br_stepena_otrovnosti)
        self.br_stepena_otrovnosti=br_stepena_otrovnosti
        self.num_agents = N
        self.num_food = num_of_food
        self.grid = MultiGrid(width,height,True)
        self.schedule= RandomActivation(self)
        self.running = True
        for i in range(self.num_agents):
            a = HungryAgent(i,self,memory_size=agent_memory_size,br_stepena_otrovnosti=br_stepena_otrovnosti,walk_energy=agent_walk_energy)
            self.schedule.add(a)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a,(x,y))
            
        for i in range(self.num_food):
            kombinacija = self.kombinacije[i]
            id_offset = i+1000
            f = FoodAgent(id_offset,self, kombinacija[0],kombinacija[1],kombinacija[2])
            self.schedule.add(f)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(f,(x,y))
            
            
        self.datacollector = DataCollector(
        model_reporters = {"TotalKnowledge":compute_knowledge,"TotalEnergy":total_energy,"TotalExperience":measure_experience,"TotalFood":total_pojedena_hrana,"TotalPoison":total_pojedeni_otrovi})
  #      agent_reporters = {"Knowledge":"knowledge"})
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
        
    def raspodeli_hranu(self,kombinacije,br_stepena_otrovnosti):
        """Od svih mogucih kombinacija hrane, on ih svrstava po otrovnosti. znaci ako imamo 64 hrane i 8 nivoa otrovnosti svaki nivo ce imati 8 stvari"""
        nivo = -br_stepena_otrovnosti//2-1 #krecemo od -
        food_dict = dict()
        broj_objekata = len(kombinacije)
        br_objekata_po_stepenu = broj_objekata//br_stepena_otrovnosti
        for i,k in enumerate(kombinacije):
            if (i%br_objekata_po_stepenu==0):
                nivo+=1
                if nivo==0:
                    nivo+=1
            assert(nivo!=0)
            food_dict[k]=nivo
        assert(len(food_dict.keys())==len(kombinacije))
        return food_dict

In [13]:
#velicina podeoka u memoriji=len(memorija)/br_stepena_otrovnosti
memory_size = 100
br_stepena_otrovnosti = 8
vel_podeoka = memory_size//br_stepena_otrovnosti

def make_insertion_dict(br_stepena_otrovnosti,vel_podeoka):
    poz = dict()
    pozicija = 0
    for i in range(-(br_stepena_otrovnosti//2),0):
        poz[i]=pozicija
        pozicija+=vel_podeoka
        poz[-i]=pozicija
        pozicija+=vel_podeoka
    return poz
        
poz = make_insertion_dict(br_stepena_otrovnosti,vel_podeoka)
print (poz)
range?
#pozicija = 0
# for i in range(len(memorija),step=4):
#    pozicije_u_proceduralnoj[otrovnost]=pozicija
# pozicija+=vel_podeoka

{-4: 0, 4: 12, -3: 24, 3: 36, -2: 48, 2: 60, -1: 72, 1: 84}


In [37]:
from collections import Counter
boje = ["red","yellow","green","blue"]
ukusi = ["kiselo","ljuto","gorko","slatko"]
oblici = ["zvezda","kvadrat","krug","trougao"]
sve_kombinacije= list(starmap(komb_tuple,product(boje,ukusi,oblici)))
def raspodeli_hranu(kombinacije,br_stepena_otrovnosti):
        """Od svih mogucih kombinacija hrane, on ih svrstava po otrovnosti. znaci ako imamo 64 hrane i 8 nivoa otrovnosti svaki nivo ce imati 8 stvari"""
        nivo = -br_stepena_otrovnosti//2 -1 #krecemo od -5, posto cemo uvecati tamo odmah
        food_dict = dict()
        broj_objekata = len(kombinacije)
        br_objekata_po_stepenu = broj_objekata//br_stepena_otrovnosti
        for i,k in enumerate(kombinacije):
            if (i%br_objekata_po_stepenu==0):
                nivo+=1
                if nivo==0:
                    nivo+=1
            assert(nivo!=0)
            food_dict[k]=nivo
        assert(len(food_dict.keys())==len(kombinacije))
        c = Counter(food_dict.values())
        for b,k in c.items():
            assert(k==br_objekata_po_stepenu)
        return food_dict
    
f = raspodeli_hranu(sve_kombinacije,br_stepena_otrovnosti=8)


In [41]:
model  = HungerModel(20,10,10)

for i in range(100):
    model.step()
    
model.datacollector.get_model_vars_dataframe()

HRANLJIVOST: 0
2
Memorija(kombinacija=Kombinacija(boja='green', ukus='slatko', oblik='zvezda'), hranljivost=2)
HRANLJIVOST: 0
-1
Memorija(kombinacija=Kombinacija(boja='green', ukus='slatko', oblik='zvezda'), hranljivost=2)
Memorija(kombinacija=Kombinacija(boja='yellow', ukus='gorko', oblik='kvadrat'), hranljivost=-1)
HRANLJIVOST: -1
HRANLJIVOST: 0
3
Memorija(kombinacija=Kombinacija(boja='blue', ukus='ljuto', oblik='zvezda'), hranljivost=3)
HRANLJIVOST: 3
1
HRANLJIVOST: 0
-3
HRANLJIVOST: 0
-3
Memorija(kombinacija=Kombinacija(boja='red', ukus='slatko', oblik='krug'), hranljivost=-3)
HRANLJIVOST: -3
HRANLJIVOST: 0
-1
Memorija(kombinacija=Kombinacija(boja='yellow', ukus='gorko', oblik='krug'), hranljivost=-1)
HRANLJIVOST: -1
HRANLJIVOST: 0
1
Memorija(kombinacija=Kombinacija(boja='green', ukus='ljuto', oblik='kvadrat'), hranljivost=1)
HRANLJIVOST: 0
-1
HRANLJIVOST: 0
1
Memorija(kombinacija=Kombinacija(boja='green', ukus='ljuto', oblik='zvezda'), hranljivost=1)
HRANLJIVOST: 1
3
HRANLJIVOST: 

Memorija(kombinacija=Kombinacija(boja='red', ukus='slatko', oblik='krug'), hranljivost=-3)
Memorija(kombinacija=Kombinacija(boja='yellow', ukus='ljuto', oblik='trougao'), hranljivost=-2)
Memorija(kombinacija=Kombinacija(boja='green', ukus='kiselo', oblik='kvadrat'), hranljivost=1)
Memorija(kombinacija=Kombinacija(boja='green', ukus='gorko', oblik='zvezda'), hranljivost=2)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='gorko', oblik='zvezda'), hranljivost=4)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='gorko', oblik='trougao'), hranljivost=4)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='gorko', oblik='kvadrat'), hranljivost=4)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='kiselo', oblik='krug'), hranljivost=3)
Memorija(kombinacija=Kombinacija(boja='green', ukus='ljuto', oblik='krug'), hranljivost=1)
Memorija(kombinacija=Kombinacija(boja='green', ukus='kiselo', oblik='zvezda'), hranljivost=1)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='ljuto', oblik='trou

Memorija(kombinacija=Kombinacija(boja='green', ukus='kiselo', oblik='zvezda'), hranljivost=1)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='ljuto', oblik='trougao'), hranljivost=3)
Memorija(kombinacija=Kombinacija(boja='yellow', ukus='gorko', oblik='krug'), hranljivost=-1)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='kiselo', oblik='zvezda'), hranljivost=3)
Memorija(kombinacija=Kombinacija(boja='green', ukus='gorko', oblik='kvadrat'), hranljivost=2)
Memorija(kombinacija=Kombinacija(boja='green', ukus='ljuto', oblik='zvezda'), hranljivost=1)
Memorija(kombinacija=Kombinacija(boja='green', ukus='slatko', oblik='zvezda'), hranljivost=2)
Memorija(kombinacija=Kombinacija(boja='yellow', ukus='gorko', oblik='kvadrat'), hranljivost=-1)
Memorija(kombinacija=Kombinacija(boja='green', ukus='kiselo', oblik='trougao'), hranljivost=1)
HRANLJIVOST: 44
4
Memorija(kombinacija=Kombinacija(boja='red', ukus='kiselo', oblik='kvadrat'), hranljivost=-4)
Memorija(kombinacija=Kombinacija(boja='red'

Memorija(kombinacija=Kombinacija(boja='green', ukus='gorko', oblik='kvadrat'), hranljivost=2)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='ljuto', oblik='krug'), hranljivost=3)
Memorija(kombinacija=Kombinacija(boja='yellow', ukus='ljuto', oblik='zvezda'), hranljivost=-2)
Memorija(kombinacija=Kombinacija(boja='yellow', ukus='slatko', oblik='trougao'), hranljivost=-1)
nadjeno u memoriji!
Memorija(kombinacija=Kombinacija(boja='red', ukus='kiselo', oblik='krug'), hranljivost=-4)
Memorija(kombinacija=Kombinacija(boja='green', ukus='slatko', oblik='zvezda'), hranljivost=2)
Memorija(kombinacija=Kombinacija(boja='yellow', ukus='gorko', oblik='kvadrat'), hranljivost=-1)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='slatko', oblik='kvadrat'), hranljivost=4)
Memorija(kombinacija=Kombinacija(boja='red', ukus='gorko', oblik='zvezda'), hranljivost=-3)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='gorko', oblik='krug'), hranljivost=4)
Memorija(kombinacija=Kombinacija(boja='blue', u

Memorija(kombinacija=Kombinacija(boja='green', ukus='ljuto', oblik='krug'), hranljivost=1)
Memorija(kombinacija=Kombinacija(boja='green', ukus='gorko', oblik='trougao'), hranljivost=2)
Memorija(kombinacija=Kombinacija(boja='yellow', ukus='slatko', oblik='krug'), hranljivost=-1)
Memorija(kombinacija=Kombinacija(boja='yellow', ukus='slatko', oblik='trougao'), hranljivost=-1)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='slatko', oblik='kvadrat'), hranljivost=4)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='gorko', oblik='zvezda'), hranljivost=4)
Memorija(kombinacija=Kombinacija(boja='green', ukus='slatko', oblik='kvadrat'), hranljivost=2)
Memorija(kombinacija=Kombinacija(boja='green', ukus='gorko', oblik='kvadrat'), hranljivost=2)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='ljuto', oblik='krug'), hranljivost=3)
Memorija(kombinacija=Kombinacija(boja='red', ukus='gorko', oblik='trougao'), hranljivost=-3)
Memorija(kombinacija=Kombinacija(boja='red', ukus='gorko', oblik='

Memorija(kombinacija=Kombinacija(boja='yellow', ukus='slatko', oblik='zvezda'), hranljivost=-1)
Memorija(kombinacija=Kombinacija(boja='green', ukus='slatko', oblik='krug'), hranljivost=2)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='gorko', oblik='zvezda'), hranljivost=4)
Memorija(kombinacija=Kombinacija(boja='red', ukus='slatko', oblik='zvezda'), hranljivost=-3)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='gorko', oblik='trougao'), hranljivost=4)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='gorko', oblik='krug'), hranljivost=4)
Memorija(kombinacija=Kombinacija(boja='green', ukus='ljuto', oblik='krug'), hranljivost=1)
Memorija(kombinacija=Kombinacija(boja='yellow', ukus='gorko', oblik='krug'), hranljivost=-1)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='kiselo', oblik='zvezda'), hranljivost=3)
Memorija(kombinacija=Kombinacija(boja='blue', ukus='kiselo', oblik='trougao'), hranljivost=3)
Memorija(kombinacija=Kombinacija(boja='green', ukus='ljuto', oblik='zvezd

,TotalKnowledge,TotalEnergy,TotalExperience,TotalFood,TotalPoison
0,0,0.0,0,0,0
1,37,-2.0,11,6,5
2,59,-17.0,16,7,9
3,85,4.0,23,13,11
4,97,11.0,29,18,13
5,111,-10.0,34,21,16
6,130,-7.0,41,24,21
7,146,7.0,48,30,23
8,158,14.0,54,36,24
9,166,-6.0,57,38,26


In [27]:
from collections import deque
class MemoryDeque(deque):
    def __init__(self,liste,maxlen):
        super().__init__(liste,maxlen)
    def insert_and_pop(self,index,what):
        try:
            self.insert(index,what)
        except IndexError:
            self.pop()
            self.insert(index,what)

        
        

In [29]:
m = MemoryDeque(["no"]*10,maxlen=10)
m.insert_and_pop(5,"yes")
m

MemoryDeque(['no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no'])

In [41]:
from itertools import product,starmap
from collections import namedtuple
boje = ["red","yellow","green","blue"]
ukusi = ["kiselo","ljuto","gorko","slatko"]
oblici = ["zvezda","kvadrat","krug","trougao"]
sve_kombinacije = list(itertools.product(boje,ukusi,oblici))
komb_tuple = namedtuple("Kombinacija","boja ukus oblik")

sve_kombinacije2= starmap(komb_tuple,product(boje,ukusi,oblici))

print(sve_kombinacije)
print(list(sve_kombinacije2))



[('red', 'kiselo', 'zvezda'), ('red', 'kiselo', 'kvadrat'), ('red', 'kiselo', 'krug'), ('red', 'kiselo', 'trougao'), ('red', 'ljuto', 'zvezda'), ('red', 'ljuto', 'kvadrat'), ('red', 'ljuto', 'krug'), ('red', 'ljuto', 'trougao'), ('red', 'gorko', 'zvezda'), ('red', 'gorko', 'kvadrat'), ('red', 'gorko', 'krug'), ('red', 'gorko', 'trougao'), ('red', 'slatko', 'zvezda'), ('red', 'slatko', 'kvadrat'), ('red', 'slatko', 'krug'), ('red', 'slatko', 'trougao'), ('yellow', 'kiselo', 'zvezda'), ('yellow', 'kiselo', 'kvadrat'), ('yellow', 'kiselo', 'krug'), ('yellow', 'kiselo', 'trougao'), ('yellow', 'ljuto', 'zvezda'), ('yellow', 'ljuto', 'kvadrat'), ('yellow', 'ljuto', 'krug'), ('yellow', 'ljuto', 'trougao'), ('yellow', 'gorko', 'zvezda'), ('yellow', 'gorko', 'kvadrat'), ('yellow', 'gorko', 'krug'), ('yellow', 'gorko', 'trougao'), ('yellow', 'slatko', 'zvezda'), ('yellow', 'slatko', 'kvadrat'), ('yellow', 'slatko', 'krug'), ('yellow', 'slatko', 'trougao'), ('green', 'kiselo', 'zvezda'), ('green'